# DATA620 Final Project A
## Network Analysis
### Euclid Zhang ~ Sam Reeves ~ David Moste

In [1]:
import pandas as pd
from pathlib import *
import json
import networkx as nx
import matplotlib as plt
import bs4

In [2]:
# Company incorporation documents
com_dir = Path('/home/s/fpa/data/company_info')

# Investment proposals and information about real projects
inv_dir = Path('/home/s/fpa/data/investment_info')

# Definitions for codes found in the above documents
#def_dir = Path('/home/s/fpa/data/definitions')

# Helper Functions
***
#### pathToList()
- Takes a directory full of JSON files
- Returns a list containing a dict for each file read

#### companyInfo()
- Takes a list of dicts
- Extracts Company Name, Addresses, Officers, and Officer Titles
- Returns a DataFrame with this information

In [3]:
def pathToList(path_obj):
    file_list = []
    for file_path in path_obj.iterdir():
        data = json.loads(file_path.read_bytes())
        file_list.append(data)
    return(file_list)

def companyInfo(com_list = com_list):
    info_list = []
    
    # Iterate over all companies in the list
    for i in range(len(com_list)):
        info = {}
        
        # Extract Company Name to "title"
        info['title'] = com_list[i]['Officers'][0]['CompanyName']
        
        # Extract Company Addresses
        for a in range(len(com_list[i]['Addresses'])):
            info['address' + str(a)] = BeautifulSoup(
                com_list[i]['Addresses'][a]['UIFormattedAddress']).get_text()
        
        # Extract Officer Information
        for b in range(len(com_list[i]['Officers'])):
            info['officer' + str(b) + 'name'] = com_list[0]['Officers'][0]['FullNameNormalized']
            info['officer' + str(b) + 'type'] = com_list[0]['Officers'][0]['OfficerType']
            
        # Convert to DataFrame
        info_list.append(info)
        df = pd.DataFrame.from_dict(info_list, orient = 'columns')
        
    return(df)

# Preprocessing
***

In [4]:
com_list = pathToList(com_dir)
inv_list = pathToList(inv_dir)

# Network Analysis
***

# Conclusions
***